In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/repos/branches/ProactiveHRI.jl`


In [2]:
using Interpolations
using MAT
using StaticArrays
using Parameters
using JuMP, HiGHS, ECOS


In [3]:
include("dynamics.jl")
include("planner.jl")
include("planner_utils.jl")
include("utils.jl")
include("plotting.jl")
include("mpc.jl")
include("sim.jl")

simulate_human_social_forces (generic function with 1 method)

In [4]:
include("velocity_obstacles.jl")

reactive_velocity_obstacles (generic function with 1 method)

In [19]:
# HJIdata = matread("../hj_cache/DynamicallyExtendedUnicycle_VO_40_40_10_12_12.mat")
HJIdata = matread("../hj_cache/DynamicallyExtendedUnicycle_VO_50_50_10_20_20.mat")

V_mat = HJIdata["V"]
V_mat = [V_mat;;;V_mat[:,:,1:1,:,:]]
grid_knots = tuple((x -> convert(Vector{Float32}, vec(x))).(HJIdata["grid_knots"])...)
push!(grid_knots[3], -grid_knots[3][1])
V = interpolate(Float32, Float32, grid_knots, V_mat, Gridded(Linear()));
# grid = HJIdata["grid"];

50×50×11×20×20 interpolate((::Vector{Float32},::Vector{Float32},::Vector{Float32},::Vector{Float32},::Vector{Float32}), ::Array{Float32, 5}, Gridded(Linear())) with element type Float32:
[:, :, 1, 1, 1] =
 -1.21421   -1.18565   -1.15772   …  -1.15772   -1.18565   -1.21421
 -1.18565   -1.15649   -1.12794      -1.12794   -1.15649   -1.18565
 -1.15772   -1.12794   -1.09877      -1.09877   -1.12794   -1.15772
 -1.13045   -1.10005   -1.07023      -1.07023   -1.10005   -1.13045
 -1.10389   -1.07285   -1.04239      -1.04239   -1.07285   -1.10389
 -1.07808   -1.0464    -1.01527   …  -1.01527   -1.0464    -1.07808
 -1.05307   -1.02074   -0.98894      -0.98894   -1.02074   -1.05307
 -1.0289    -0.995925  -0.963444     -0.963444  -0.995925  -1.0289
 -1.00564   -0.972004  -0.938841     -0.938841  -0.972004  -1.00564
 -0.983322  -0.949036  -0.91519      -0.91519   -0.949036  -0.983322
  ⋮                               ⋱                        
 -1.00564   -0.972004  -0.938841     -0.938841  -0.9720

In [20]:
dt = 0.1
velocity_max = 2.0
robot = DynamicallyExtendedUnicycle(dt, velocity_max, [1., 3.])


DynamicallyExtendedUnicycle{Float64}
  dt: Float64 0.1
  state_dim: Int64 4
  ctrl_dim: Int64 2
  velocity_min: Float64 0.0
  velocity_max: Float64 2.0
  control_min: Array{Float64}((2,)) [-1.0, -3.0]
  control_max: Array{Float64}((2,)) [1.0, 3.0]


In [22]:
# robot_states, robot_controls, human_states, human_controls
i = 59
state = robot_states[i,:]
other_position = get_position(human, human_states[i,:])
other_velocity = get_velocity(human, human_states[i,:], human_controls[i,:])[:]
desired_control = ego_desired_controls[i]

UndefVarError: UndefVarError: `robot_states` not defined

In [21]:
# state = [0.; 0.; 0.; 1.0]
# state = [1.8799899310399428, 0.06433319244684152, -0.026440947318282934, 0.8319310568830395]
# other_position = [3.; 3.]
# # other_velocity = [-1.; -1.3]
# other_velocity = [-1.499997640038893, 0.00011934669432259218]
# desired_control = [0.5; 1.]
rel_state = relative_state(robot, state, other_position, other_velocity)
VO_QP = construct_VO_QP_base(robot, desired_control)
tol = 1E-2

# computing reactive control
u_out, ϵ, g, f0, B, V_min = reactive_velocity_obstacles(VO_QP, 
                            robot,
                            state, 
                            desired_control,
                            [other_position],
                            [other_velocity], tol=tol)
u_out, ϵ, V_min

UndefVarError: UndefVarError: `state` not defined

## Plotting stuff

In [ ]:
l = @layout [a b]

plot_value = plot(aspect_ratio=:equal, size=(1500,800), title="$(round(V_min, digits=4))", top_margin=-10mm, grid=true)
value_function = transpose(V.([grid_knots[1]], [grid_knots[2]], Ref(rel_state[3]), Ref(rel_state[4]), Ref(rel_state[5]))[1])
contourf!(plot_value, grid_knots[1], grid_knots[2], value_function, alpha=0.3)
contour!(plot_value, grid_knots[1], grid_knots[2], value_function, levels=[tol], linewidth=2, color=:black)
scatter!(plot_value, [0.], [0.], color=:blue, markersize=5, label="ego")
quiver!(plot_value, [0.], [0.], quiver=([rel_state[5]], [0]), linewidth=2, color=:blue)
scatter!(plot_value, [rel_state[1]], [rel_state[2]], color=:red, markersize=5, label="other")
quiver!(plot_value, [rel_state[1]], [rel_state[2]], quiver=([rel_state[4] * cos(rel_state[3])], [rel_state[4] * sin(rel_state[3])]), linewidth=2, color=:red)

lower = robot.control_min
upper = robot.control_max

plot_controls = plot(aspect_ratio=:equal, xlabel="ω", ylabel="a", xlim=(lower[1]-0.1, upper[1]+0.1), ylim=(lower[2]-0.1, upper[2]+0.1), bottom_margin=10mm, right_margin=5mm, top_margin=5mm)
c = dot(g, f0)
a = dot(g,B[:,1])
b = dot(g,B[:,2]) 
@show a, b, c

if V_min < tol
    if  (abs(b) > 1E-6) & (abs(a) > 1E-6)
        println("diagonal")
        y = [-c-a*lower[1], -c-a* upper[1]] / b
        if (b > 0)
            plot!(plot_controls, [lower[1],upper[1]], y, fillrange=[lower[2], lower[2]], fillalpha=0.6, color=:pink, label="")
        else
            plot!(plot_controls, [lower[1],upper[1]], y, fillrange=[upper[2], upper[2]], fillalpha=0.6, color=:pink, label="")
        end
    elseif (abs(b) < 1E-6) & (abs(a) > 1E-6)
        println("vertical")
        if a > 0  # u1 >= RHS
            x = min(-c/a, upper[1])
            vspan!(plot_controls, [lower[1], -c/a], alpha=0.6, color=:pink, label="unsafe")
            plot!(plot_controls, [-c/a, -c/a], [lower[2], upper[2]], label="")
        else # u1 <= RHS
            x = max(-c/a, lower[1])
            vspan!(plot_controls, [-c/a, upper[1]], alpha=0.6, color=:pink, label="unsafe")
            plot!(plot_controls, [-c/a, -c/a], [lower[2], upper[2]], label="")
        end
    elseif (abs(b) > 1E-6) & (abs(a) < 1E-6)
        println("horizontal")
        if b > 0
            plot!(plot_controls, [lower[1], upper[1]], [-c/b, -c/b], fillrange=[lower[2], lower[2]], fillalpha=0.6, color=:pink, label="")
        else
            plot!(plot_controls, [lower[1], upper[1]], [-c/b, -c/b], fillrange=[upper[2], upper[2]], fillalpha=0.6, color=:pink, label="")
        end
    end
end

plot!(plot_controls, [lower[1],upper[1]], [lower[2], lower[2]], color=:black, label="")
plot!(plot_controls, [upper[1],upper[1]], [lower[2], upper[2]], color=:black, label="")
plot!(plot_controls, [lower[1],upper[1]], [upper[2], upper[2]], color=:black, label="")
plot!(plot_controls, [lower[1],lower[1]], [lower[2], upper[2]], color=:black, label="")
scatter!(plot_controls, [desired_control[1]], [desired_control[2]], markersize=7, color=:green, label="desired")
scatter!(plot_controls, [u_out[1]], [u_out[2]], markersize=7, color=:blue, label="computed")

plot(plot_value, plot_controls, layout = l)

# Wrapping HJ into sim

In [23]:
function simulate_hj(ego_hps::PlannerHyperparameters, other_ip::InteractionPlanner, ego_initial_state::Vector{Float64}, ego_goal_state::Vector{Float64}, sim_horizon::Int64; ibr_iterations=3::Int64, leader="ego"::String, verbose=false::Bool)
    # Given the IP problem setup of the ego agent and other agent
    # initialize matrices for saving the paths

    # HJIdata = matread("../hj_cache/DynamicallyExtendedUnicycle_VO_40_40_10_12_12.mat")
    HJIdata = matread("../hj_cache/DynamicallyExtendedUnicycle_VO_50_50_10_20_20.mat")

    V_mat = HJIdata["V"]
    V_mat = [V_mat;;;V_mat[:,:,1:1,:,:]]
    grid_knots = tuple((x -> convert(Vector{Float32}, vec(x))).(HJIdata["grid_knots"])...)
    push!(grid_knots[3], -grid_knots[3][1])
    global V = interpolate(Float32, Float32, grid_knots, V_mat, Gridded(Linear()));
    print(typeof(V))

    ego_dyn = ego_hps.dynamics
    other_dyn = other_ip.ego_planner.incon.hps.dynamics

    ego_traj = Vector{Vector{Float64}}(undef, sim_horizon + 1)
    ego_controls = Vector{Vector{Float64}}(undef, sim_horizon)
    other_traj = Vector{Vector{Float64}}(undef, sim_horizon + 1)
    other_controls = Vector{Vector{Float64}}(undef, sim_horizon)
    ego_desired_controls = Vector{Vector{Float64}}(undef, sim_horizon)

    ego_traj[1] = ego_initial_state
    other_traj[1] = other_ip.ego_planner.incon.opt_params.initial_state

    ego_solve_times = Vector{Float64}(undef, sim_horizon)
    other_solve_times = Vector{Float64}(undef, sim_horizon)

    VO_QP = construct_VO_QP_base(ego_dyn, [0.; 0.])

    # Uses MPC function to simulate to a given time horizon
    for i in 1:(sim_horizon)

        ego_state = ego_traj[i]
        other_state = other_traj[i]
        # solve for the next iteration

        ego_opt_params_ = PlannerOptimizerParams(ego_dyn, ego_hps, ego_state, ego_goal_state, "ECOS")
        ego_ideal_planner_ = IdealProblem(ego_dyn, ego_hps, ego_opt_params_)
        solve(ego_ideal_planner_)
        ego_desired_control = ego_ideal_planner_.opt_params.previous_controls[1, :][1]

        other_position = get_position(other_dyn, other_state)
        if i == 1
            other_velocity = get_velocity(other_dyn, other_state, zeros(Float64, other_dyn.ctrl_dim))[:]
        else
            other_velocity = get_velocity(other_dyn, other_state, other_controls[i-1])[:]
        end
        
        rel_state = relative_state(ego_dyn, ego_state, other_position, other_velocity)

        ego_control, ϵ, ∇V, f0, B, V_min= minimally_invasive_velocity_obstacles(VO_QP, 
        ego_dyn,
        ego_state, 
        ego_desired_control,
        [other_position],
        [other_velocity]
)

        other_control = mpc_step(other_ip, other_state, ego_state, ibr_iterations=ibr_iterations, leader=leader)

        ego_state = step(ego_dyn, ego_state, ego_control)
        other_state = step(other_dyn, other_state, other_control)

        ego_traj[i+1] = ego_state
        other_traj[i+1] = other_state
        ego_controls[i] = ego_control
        other_controls[i] = other_control
        ego_desired_controls[i] = ego_desired_control

    end

    # cast vector of vectors to matrix for easier plotting
    ego_traj = vector_of_vectors_to_matrix(ego_traj)
    ego_controls = vector_of_vectors_to_matrix(ego_controls)
    other_traj = vector_of_vectors_to_matrix(other_traj)
    other_controls = vector_of_vectors_to_matrix(other_controls)

    ego_traj, ego_controls, other_traj, other_controls, ego_desired_controls
end

simulate_hj (generic function with 1 method)

In [24]:
test_robot = DynamicallyExtendedUnicycle(0.1, 2., [1., 0.5])

DynamicallyExtendedUnicycle{Float64}
  dt: Float64 0.1
  state_dim: Int64 4
  ctrl_dim: Int64 2
  velocity_min: Float64 0.0
  velocity_max: Float64 2.0
  control_min: Array{Float64}((2,)) [-1.0, -0.5]
  control_max: Array{Float64}((2,)) [1.0, 0.5]


In [25]:
test_robot_hps = PlannerHyperparameters(test_robot)

PlannerHyperparameters{Float64}
  dynamics: DynamicallyExtendedUnicycle{Float64}
  time_horizon: Int64 20
  Q: Array{Float64}((4, 4)) [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
  R: Array{Float64}((2, 2)) [1.0 0.0; 0.0 1.0]
  Qt: Array{Float64}((4, 4)) [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
  markup: Float64 1.0
  collision_slack: Float64 1000.0
  trust_region_weight: Float64 10.0
  inconvenience_weights: Array{Float64}((1, 3)) [1.0 1.0 1.0]
  collision_radius: Float64 0.25
  inconvenience_ratio: Float64 0.1


In [26]:
robot_states, robot_controls, human_states, human_controls, ego_desired_controls = simulate_hj(test_robot_hps, human_ip, [0.; 0.1; 0.; 0.5], [9.; 0.; 0.; 0.5], 60)

Interpolations.GriddedInterpolation{Float32, 5, Array{Float32, 5}, Gridded{Linear{Throw{OnGrid}}}, NTuple{5, Vector{Float32}}}

([0.0 0.1 0.0 0.5; 0.05127979981451732 0.09990176158346324 -0.003799847580839568 0.525598495164839; … ; 2.2896325873240877 -0.10702025125588455 0.027290985715033033 0.05208035117128622; 2.2943598012067756 -0.10689120889092182 0.027171406238704374 0.042499145601814024], [-0.03799847580839568 0.2559849516483903; -0.03308223432362251 0.23165129244454605; … ; -0.0008109732374446121 -0.09930706226187203; -0.0011957947632865729 -0.09581205569472195], [9.0 0.0 3.141592653589793 0.0; 8.99250007494746 9.184759209438255e-19 3.141598282582999 0.1499985010508016; … ; 1.251019425432554 -0.33173377474695814 3.010106804831487 1.0750157893887073; 1.1464101268999083 -0.31701617979156477 2.9934332628133147 1.0377995321649465], [5.6289932059380315e-5 1.499985010508016; 0.04036090884543168 1.4999906379357013; … ; -0.17954152793619832 -0.3675692301534539; -0.16673542018172438 -0.3721625722376087], [[-0.03799847580839568, 0.2559849516483903], [-0.03308223432362251, 0.23165129244454605], [-0.0288802553122904

In [ ]:
# using Serialization
# interesting_scenario = (robot_states, robot_controls, human_states, human_controls, ego_desired_controls)
# serialize("../interesting_sit/hj_wiggly.dat", interesting_scenario)

In [ ]:
animation(robot_states, human_states, pos_xlims=[-1, 11])

In [13]:
solver = "ECOS"
dt = 0.1
velocity_max = 1.5
human = DynamicallyExtendedUnicycle(dt, velocity_max, [1., 1.5])

time_horizon = 25
Q = diagm([0.0; 0.0; 0.; 0.])
R = diagm([1.; 0.3]) 
Qt = diagm([10.; 10.; 0.; 0.])
markup = 1.05
collision_slack = 50.
trust_region_weight = 5.
inconvenience_weights = [1.; 1.; 0.1]
collision_radius = 1.
inconvenience_ratio = 0.2


human_hps = PlannerHyperparameters(dynamics=human,
                             time_horizon=time_horizon,
                             Q=Q,
                             R=R,
                             Qt=Qt,
                             markup=markup,
                             collision_slack=collision_slack,
                             trust_region_weight=trust_region_weight,
                             inconvenience_weights=inconvenience_weights,
                             collision_radius=collision_radius,
                             inconvenience_ratio=inconvenience_ratio)



dt = 0.1
velocity_max = 1.5
# robot = Unicycle(dt, velocity_max, [1.0, 2.])
robot = DynamicallyExtendedUnicycle(dt, velocity_max, [1., 1.5])

# time_horizon = 45
Q = diagm([0.0; 0.0; 0.; 0.])
R = diagm([1.; 0.0]) 
Qt = diagm([10.; 10.; 0.; 0.])

robot_hps = PlannerHyperparameters(dynamics=robot,
                             time_horizon=time_horizon,
                             Q=Q,
                             R=R,
                             Qt=Qt,
                             markup=markup,
                             collision_slack=collision_slack,
                             trust_region_weight=trust_region_weight,
                             inconvenience_weights=inconvenience_weights,
                             collision_radius=collision_radius,
                             inconvenience_ratio=inconvenience_ratio)

robot_initial_state = [1.; -2.; 0.; 0.]
robot_goal_state = [9.; 0.; 0.; 0.]
human_initial_state = [9.; 0.; pi; 0.]
human_goal_state = [0.; 0.; π; 0.]
# setting up the IP object to be serialized and saved for all trials
human_ip = InteractionPlanner(human_hps, 
                        robot_hps,
                        human_initial_state,
                        robot_initial_state,
                        human_goal_state,
                        robot_goal_state,
                        solver)

InteractionPlanner
  ego_planner: AgentPlanner
  other_planner: AgentPlanner


In [ ]:
test_robot = DynamicallyExtendedUnicycle(0.1, 2., [1., 2.])
test_hps = PlannerHyperparameters(test_robot)
test_opt_params = PlannerOptimizerParams(test_robot, test_hps, [0.; 0.; 0.; 0.], [10.; 0.; 0.; 0.], "ECOS")

In [ ]:
test_prob = IdealProblem(test_robot, test_hps, test_opt_params)

In [ ]:
solve(test_prob)
test_prob.opt_params.previous_controls

In [ ]:
test_prob.opt_params.previous_states

In [ ]:
plot_solve_solution(test_prob)